# Difference in Difference

The data is available at this [URL](https://github.com/KubiaPXH/Memoire-M1/raw/master/data/final_air_pollution_30cities.csv)

need to install lfe: https://cran.r-project.org/web/packages/lfe/lfe.pdf

## ressources

If you need extra doc for R:

- https://drive.google.com/open?id=1NM_oGxntXh2b0p5mv_oYlkMSx-aXk2Rg
- https://drive.google.com/open?id=1Waxak8_PR7VhIwynWoegrVtHn1_Tm2i-
- https://drive.google.com/open?id=1RokSoGrwQgwiJfm842Fw5oPhCudWfwrs
- https://drive.google.com/open?id=1ADSij1macPea9QhUtzrIWNgQBuJybWRm
- https://drive.google.com/open?id=1rs1JZcElwHnKWex8b2c_F49Vxzc3ukfp
- https://drive.google.com/open?id=1x77KTmsVA7Q1IFji4gALv1dhNfIvTplc

In [ ]:
library(tidyverse)
library(lfe)

We get the data from Github and then we perform the following steps:

- import the data with the correct types
- covert character to factor
- Make sure of the level reference

In [ ]:
url <- paste("https://raw.githubusercontent.com/KubiaPXH/Memoire-M1/",
"master/data/final_air_pollution_30cities.csv",  sep="")

In [ ]:
df_ <- read_csv(url, col_types = cols(
  city = col_character(),
  year = col_character(),
  PM10 = col_double(),
  SO2 = col_double(),
  NOx = col_double(),
  pop_density = col_double(),
  GRP_pc = col_double(),
  second_industry = col_double(),
  gas_supply = col_double(),
  green_coveraged = col_double(),
  post = col_character(),
  key_regions = col_character())
               )  %>% 
mutate_if(is.character, as.factor) %>%
mutate(
    post = relevel(post, ref='0'),
    key_regions = relevel(key_regions, ref='0'),
    ## For explanation purpose. Kunming is not in key region
    city = relevel(city, ref='Kunming')
    
    )

In [ ]:
glimpse(df_)

In [ ]:
colnames(df_)[ apply(df_, 2, anyNA)] 

In [ ]:
#df_ %>% group_by(city) %>% summarize(length(city))

In [ ]:
#df_ %>% group_by(city, key_regions) %>% summarize(length(city))

In [ ]:
#df_ %>% group_by(city, post) %>% summarize(length(city))

In [ ]:
df_ %>% 
group_by(city, key_regions, post) %>% 
summarize(avg_pm = mean(log(PM10))) %>% 
spread(post, avg_pm, sep = "")%>%
mutate(diff  = post1 -post0 ) %>%
arrange(diff) %>%
ggplot(aes(x=reorder(city, -diff),y=diff, fill = key_regions))+ 
geom_bar(stat="identity")+ 
theme_classic()+ 
coord_flip()+ 
labs( x="City",
     y="Difference PM", 
     title =paste( "Difference of average Log of PM before and after the Policy" ) )

In [ ]:
summary(df_)

# Define the model


## Test without FE

$$ \text{ln PM 10}_{ci} = \alpha Post + \beta \text{ key region}+ \alpha Post \times \text{key region} + \epsilon_{ct}$$

For the sake of the tutorial, we don't cluster the standard error -> We want to make sure we get the same results 

In [ ]:
df_ %>% summarize( mean(log(PM10)))

In [ ]:
model <- log(PM10) ~ post +  key_regions + post:key_regions
fit <- lm(model, df_)
summary(fit)

In [ ]:
model <- log(PM10) ~ post * key_regions 
fit <- lm(model, df_)
summary(fit)

With the FE library

In [ ]:
summary(felm(formula=log(PM10) ~ post +  key_regions + post : key_regions  |
             FALSE | 0 |FALSE, data = df_, exactDOF=TRUE))

In [ ]:
summary(felm(formula=log(PM10) ~  post * key_regions  |
             FALSE | 0 |FALSE, data = df_, exactDOF=TRUE))

## Test with FE

We add city fixed effect. It leads therefore to singularities with the interaction term. We should expect that the coefficient `key_regions1` to be dropped 

$$ \text{ln PM 10}_{ci} = \alpha Post + \alpha Post \times \text{key region}+ \gamma_c + \epsilon_{ct}$$

For the sake of the tutorial, we don't cluster the standard error -> We want to make sure we get the same results 



In [ ]:
summary(lm(log(PM10) ~ post + key_regions + post:key_regions +city, df_))

In [ ]:
summary(lm(log(PM10) ~ post*key_regions+city, df_))

In [ ]:
4.14993 - 4.21276

In [ ]:
df_ <- read_csv(url, col_types = cols(
  city = col_character(),
  year = col_character(),
  PM10 = col_double(),
  SO2 = col_double(),
  NOx = col_double(),
  pop_density = col_double(),
  GRP_pc = col_double(),
  second_industry = col_double(),
  gas_supply = col_double(),
  green_coveraged = col_double(),
  post = col_character(),
  key_regions = col_character())
               )  %>% 
mutate_if(is.character, as.factor) %>%
mutate(
    post = relevel(post, ref='1'),
    key_regions = relevel(key_regions, ref='0'),
    ## For explanation purpose. Kunming is not in key region
    city = relevel(city, ref='Kunming')
    
    )
summary(lm(log(PM10) ~ post + post:key_regions+city - 1, df_))

In [ ]:
df_ <- read_csv(url, col_types = cols(
  city = col_character(),
  year = col_character(),
  PM10 = col_double(),
  SO2 = col_double(),
  NOx = col_double(),
  pop_density = col_double(),
  GRP_pc = col_double(),
  second_industry = col_double(),
  gas_supply = col_double(),
  green_coveraged = col_double(),
  post = col_character(),
  key_regions = col_character())
               )  %>% 
mutate_if(is.character, as.factor) %>%
mutate(
    post = relevel(post, ref='0'),
    key_regions = relevel(key_regions, ref='0'),
    ## For explanation purpose. Kunming is not in key region
    city = relevel(city, ref='Kunming')
    
    )

summary(felm(formula=log(PM10) ~  post * key_regions  |
             city | 0 |FALSE, data = df_, exactDOF=TRUE))

## Add control

$$ \text{ln PM 10}_{ci} = \alpha Post + \beta \text{ key region}+ \alpha Post \times \text{key region} + X_{ct} + \gamma_c + \epsilon_{ct}$$

In [ ]:
summary(felm(formula=log(PM10) ~ post *  key_regions + 
             log(pop_density)|
             city| 0 |FALSE, data = df_, exactDOF=TRUE))

In [ ]:
summary(felm(formula=log(PM10) ~ post *  key_regions + 
             log(pop_density) + log(GRP_pc)|
             city| 0 |FALSE, data = df_, exactDOF=TRUE))

In [ ]:
summary(felm(formula=log(PM10) ~ post *  key_regions + 
             log(pop_density) + log(GRP_pc) + log(second_industry)|
             city| 0 |FALSE, data = df_, exactDOF=TRUE))